# Extract Noise Embedding

## Get the N-best Transcriptions

In [1]:
!pip install git+https://github.com/johnlohjy/whisper.git@Nbest_transcriptions
!pip install datasets

  Cloning https://github.com/johnlohjy/whisper.git (to revision Nbest_transcriptions) to /tmp/pip-req-build-juf4ccvq
  Running command git clone --filter=blob:none --quiet https://github.com/johnlohjy/whisper.git /tmp/pip-req-build-juf4ccvq
  Running command git checkout -b Nbest_transcriptions --track origin/Nbest_transcriptions
  Switched to a new branch 'Nbest_transcriptions'
  Branch 'Nbest_transcriptions' set up to track remote branch 'Nbest_transcriptions' from 'origin'.
  Resolved https://github.com/johnlohjy/whisper.git to commit 7b1c525cb96e5d846973458fa4c4bda74bf79844
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=809412 sha256=6295eecef8fbb53091

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
# Load the dataset
# gigapseech takes about 5mins
from datasets import load_dataset
ds_train = load_dataset("speechcolab/gigaspeech", "xs")

The repository for speechcolab/gigaspeech contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/speechcolab/gigaspeech.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


data/xs_n_archives.txt:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

data/test_n_archives.txt:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

xs_chunks_0000.tar.gz:   0%|          | 0.00/972M [00:00<?, ?B/s]

dev_chunks_0000.tar.gz:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

...

test_chunks_0001_metadata.csv:   0%|          | 0.00/2.66M [00:00<?, ?B/s]

test_chunks_0002_metadata.csv:   0%|          | 0.00/1.49M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
import whisper

model = whisper.load_model("base")
result = model.transcribe(ds_train["train"][0]["audio"]["path"], beam_size= 10)

def retrieve_n_best_transcriptions(result):
  n_best = []
  for d in result:
    n_best.append(d['text'])
  return n_best

n_best = retrieve_n_best_transcriptions(result)

for text in n_best:
  print(text)

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 232MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

 As you're leaving, can cache pulls RSI really quickly?
 As you're leaving, can cache pulls RSI really quickly?
 As you're leaving, can cache pulls RSI really quickly?
 As you're leaving, can cache pulls RSI really quickly?
 As you're leaving, can cache pulls RSI really quickly?
 As you're leaving, can cache pulls RSI really quickly?
 As your leaving can cache pulls RSI really quickly.
 As your leaving can cache pulls RSI really quickly.
 As your leaving can cache pulls RSI really quickly.
 As your leaving can cache pulls RSI really quickly.


## Lang-Space Noise Embedding 1: Extract sentence-level noise embedding

Dimension of Lang-Space Noise Embedding 1 (utterance-level) according to RobustGER:

```
(Nx(N-1)/2) x Dim of Sbert
```

In [6]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 7.2 MB/s eta 0:00:00


In [7]:
"""
Step 1:
For each transcription i,
  Compute the pairwise sentence embedding differences with transcriptions j such that j > i using the sentence-BERT encoder

Step 2:
Concatenate all these sentence-level noise embeddings

SBERT documentation
- https://sbert.net/
- https://huggingface.co/sentence-transformers
- https://towardsdatascience.com/sbert-deb3d4aef8a4
"""

# Load a pretrained Sentence Transformer model
# diff model specs: https://sbert.net/docs/sentence_transformer/pretrained_models.html
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")
sbert_embeddings = []
for transcription in n_best:
  sbert_embeddings.append(model.encode(transcription))


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
sbert_embeddings

[array([ 1.33149803e-03,  4.79083322e-03, -4.79827262e-02, -1.01084926e-03,
         9.07631144e-02, -6.25970289e-02, -2.41307002e-02, -5.96474856e-02,
         8.53897538e-03, -4.06354368e-02, -2.91601680e-02,  1.42730236e-01,
        -3.83272953e-02,  8.00285023e-03,  6.94402978e-02, -4.55767065e-02,
         5.23186624e-02, -1.39796315e-02, -9.26286876e-02, -2.41847094e-02,
        -8.35723709e-03, -2.90318318e-02, -2.78717335e-02, -4.09430824e-02,
         2.57125590e-02, -3.54971066e-02, -3.47783267e-02,  3.23161785e-03,
         3.26427408e-02,  1.32570975e-02,  1.15862833e-02, -1.77617352e-02,
        -6.55542538e-02, -6.49554804e-02, -2.62173098e-02,  9.34168473e-02,
        -1.01479022e-02, -5.69908833e-03,  8.75133723e-02, -5.93386777e-03,
         4.27901233e-03, -9.94388107e-03, -6.43026009e-02, -1.83444172e-02,
         8.63545090e-02, -1.07604843e-02,  1.17821516e-02, -3.06848325e-02,
        -3.02058980e-02, -8.34387094e-02, -4.64843810e-02,  1.32943513e-02,
        -7.7

In [22]:
len(sbert_embeddings)

10

In [31]:
sbert_embeddings_diff = []
for i in range(len(sbert_embeddings)-1): # 0 1 2 3 4 5 6 7 8 9. We need to stop at index 8 for comparison
  for j in range(i+1,len(sbert_embeddings)):
    #print("i: " + str(i))
    #print("j: " + str(j))
    #print("")
    sbert_embeddings_diff.append(sbert_embeddings[i]-sbert_embeddings[j])

In [32]:
sbert_embeddings_diff

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 

In [33]:
import numpy as np

In [34]:
sbert_embeddings_diff = np.array(sbert_embeddings_diff)

In [35]:
# 9+8+7+6+5+4+3+2+1 comparisons
sbert_embeddings_diff.shape

(45, 384)

## Lang-Space Noise Embedding 2: Extract token-level noise embedding